In [3]:

import sys
sys.path.append("././quaos")
from math import gcd

from symplectic import PauliSum, PauliString, Pauli
import numpy as np
from gates import Circuit, SUM as CX, Hadamard as H, PHASE as S, SWAP
from circuit_utils import symplectic_effect, random_clifford
from hamiltonian import random_pauli_hamiltonian, cancel_pauli, symplectic_pauli_reduction, pauli_reduce

Identify $V$ in $H = H_0 + V$ such that $\exists$ a Clifford circuit $C$ where $II \cdots IZ$ is a symmetry of $CH_0C^\dagger$

$V = \sum_i c_i Q_i$ with $Q_i = ***\cdots **X$ are all Paulis in $CHC^\dagger$ with a leading $X$.

We want to find $C$.

In [ ]:

def solve_modular_linear(x: int, z: int, d: int) -> int:
    """
    Find smallest non-negative integer n such that (x + z*n) % d == 0.
    Returns None if no solution exists
    """
    x, z, d = int(x), int(z), int(d)
    g = gcd(z, d)
    if x % g != 0:
        raise ValueError(f"No solution of (x + z*n) % d == 0 exists for x ={x}, z ={z}, d ={d}")

    # Reduce the equation modulo d // g
    z_ = z // g
    d_ = d // g
    x_ = (-x // g) % d_

    # Use built-in pow to compute modular inverse
    try:
        z_inv = pow(z_, -1, d_)
    except ValueError:
        raise ValueError(f"No solution of (x + z*n) % d == 0 exists for x ={x}, z ={z}, d ={d}")

    n = (x_ * z_inv) % d_
    return n

def to_ix(pauli_string: PauliString, target_index: int, ignore: int | list[int] | None = None) -> Circuit:
    """Finds a circuit to turn a PauliString to III...IXI...II where the X is at target_index
    
    for qudits X = xrz0 for any r != 0, I = x0z0

    ignore is a list of qudits to not try to turn into an I
    """

    if ignore is None:
        ignore = []
    if isinstance(ignore, int):
        ignore = [ignore]
    if target_index in ignore:
        raise Exception("target_index must not be in ignore")

    p_string_in = pauli_string.copy()
    # First we make the target qudit an X
    circuit = to_x(pauli_string, target_index, ignore)
    pauli_string = circuit.act(p_string_in)
    n_q = pauli_string.n_qudits()
    for q in range(n_q):
        if q != target_index and q not in ignore:
            if pauli_string[q].x_exp == 0 and pauli_string[q].z_exp != 0:
                circuit.add_gate(H(q, pauli_string.dimensions[q]))
                pauli_string = circuit.act(p_string_in)
            if pauli_string[q].x_exp != 0:
                if pauli_string[q].z_exp != 0:
                    # use the x to cancel the z of q with S gates
                    n_s = solve_modular_linear(pauli_string[q].x_exp, pauli_string[q].z_exp, pauli_string.dimensions[q])
                    for i in range(n_s):
                        circuit.add_gate(S(q, pauli_string.dimensions[q]))
                    pauli_string = circuit.act(p_string_in)

                # use cnot to cancel the x of q with the x of target. n_cnot = n where x_q + x+_target) % d= 0
                n_cnot = solve_modular_linear(pauli_string[q].x_exp, pauli_string[target_index].x_exp, pauli_string.dimensions[target_index])
                if n_cnot is None:  
                    raise Exception("Weird")
                for i in range(n_cnot):
                    circuit.add_gate(CX(target_index, q, pauli_string.dimensions[target_index]))
                pauli_string = circuit.act(p_string_in)

    else:
        return circuit

def to_x(pauli_string: PauliString, target_index: int, ignore: int | list[int] | None = None) -> Circuit:
    """Finds a circuit to turn a PauliString to ****X*** where the X is at target_index"""
    if ignore is None:
        ignore = []
    if isinstance(ignore, int):
        ignore = [ignore]
    if target_index in ignore:
        raise Exception("target_index must not be in ignore")

    if target_index < 0:
        target_index += pauli_string.n_qudits()
    if target_index > pauli_string.n_qudits():
        raise Exception(f"target_index {target_index} out of range {pauli_string.n_qudits()}")

    n_q = pauli_string.n_qudits()
    if PauliString.n_identities == n_q:
        raise Exception("PauliString is identity - cannot be converted to X")

    circuit = Circuit(dimensions=pauli_string.dimensions)  # Empty circuit of correct dimensions
    
    # First we check if we can get there from a single qudit gate
    # target is X
    dim_target = pauli_string.dimensions[target_index]
    if pauli_string[target_index].x_exp != 0 and pauli_string[target_index].z_exp == 0:  # already x
        return circuit
    elif pauli_string[target_index].x_exp != 0 and pauli_string[target_index].z_exp != 0:
        # We remove the Z by performing S gates
        n_s_gates = solve_modular_linear(pauli_string[target_index].z_exp, pauli_string[target_index].x_exp, dim_target)
        for i in range(n_s_gates):
            circuit.add_gate(S(target_index, dim_target))  # xrzs -> xrz(s+r) until s+r = 0
        return circuit
    elif pauli_string[target_index].x_exp == 0 and pauli_string[target_index].z_exp != 0:
        circuit.add_gate(H(target_index, dim_target)) # swap x and z
        return circuit
    elif pauli_string[target_index].x_exp == 0 and pauli_string[target_index].z_exp == 0: # must be x0z0
        # We have to use a multi-qubit gate - we loop through the qudits until we find a resource
        for q in range(n_q - 1, -1, -1):
            if q != target_index and q not in ignore:
                if pauli_string[q].x_exp != 0:
                    # move x of q to target
                    circuit.add_gate(CX(q, target_index, dim_target))
                    return circuit
                if pauli_string[q].x_exp == 0 and pauli_string[q].z_exp != 0:
                    # move z of q to target_index then Hadamard
                    circuit.add_gate(CX(target_index, q, dim_target))
                    circuit.add_gate(H(target_index, dim_target))
                    return circuit
    else:
        raise Exception("Something weird is going on, a case has been missed from the above loop")
    raise Exception(f"No circuit found to convert {pauli_string} pauli to X")
    
def transform_to_basis(pauli_sum: PauliSum, indexes_already_in_basis: list[int]) -> tuple[Circuit, PauliSum]:
    circuit = Circuit(dims=pauli_sum.dimensions)
    basis_elements_max = 2 * pauli_sum.n_qudits() - len(indexes_already_in_basis)
    if basis_elements_max > pauli_sum.n_paulis():
        basis_elements_max = pauli_sum.n_paulis()
    
    for pauli_index in range(basis_elements_max):
        for qudit_index in range(pauli_sum.n_qudits()):
            if pauli_index not in indexes_already_in_basis:
                circuit = to_x(pauli_sum[pauli_index], pauli_index, indexes_already_in_basis)
                pauli_sum = circuit.act(pauli_sum)
    return circuit, pauli_sum


def find_leading_x(pauli_sum: PauliSum) -> tuple[list[int], int]:
    leading_xs = []
    if np.any(pauli_sum.x_exp):
        for i, p in enumerate(pauli_sum.pauli_strings):
            xs = p.x_exp
            if np.any(xs):
                leading_x = len(xs) - 1 - np.argmax(xs[::-1] != 0)  # furthest x to right hand side
            else:
                leading_x = None
            leading_xs.append(leading_x)
        lx_tuple = [(i, v) for i, v in enumerate(leading_xs) if v is not None]
        max_val = max(v for _, v in lx_tuple)
        indices = [i for i, v in lx_tuple if v == max_val]
        return indices, int(max_val)
    else:
        print('no leading x')
        return [], -1


def lowest_z_only(pauli_sum: PauliSum) -> int:
    z_pauli = 0
    for i in range(pauli_sum.n_paulis()):
        min_zs = pauli_sum.n_qudits()
        if pauli_sum[i].is_z():
            if sum(pauli_sum[i].z_exp) < min_zs:
                min_zs = sum(pauli_sum[i].z_exp)
                z_pauli = i
    return z_pauli

def lowest_x_only(pauli_sum: PauliSum) -> int:
    x_pauli = 0
    for i in range(pauli_sum.n_paulis()):
        min_xs = pauli_sum.n_qudits()
        if pauli_sum[i].is_x():
            if sum(pauli_sum[i].x_exp) < min_xs:
                min_xs = sum(pauli_sum[i].x_exp)
                x_pauli = i
    return x_pauli


def inv_fourier_sum_fourier(dimensions: list[int], fourier_index: int, control: int, target: int) -> Circuit:
    """
        If fourier_index=0 control=0, target=1 this is a CZ
        This has the nice effect of:

        x0zs xrz0 -> x0z{r+s} xrz0
        xr'z0 ... -> xr'z0 ...

        If fourier_index=1 control=0, target=1
        This has the nice effect of:
        
    """
    c = Circuit(dimensions)
    c.add_gate(H(fourier_index, dimensions[fourier_index], inverse=True))
    c.add_gate(CX(control, target, dimensions[control]))
    c.add_gate(H(fourier_index, dimensions[fourier_index]))
    return c

def random_pauli_string(dimensions: list[int]) -> PauliString:
    """Generates a random PauliString of n_qudits"""
    n_qudits = len(dimensions)
    x_array = np.zeros(n_qudits, dtype=int)
    z_array = np.zeros(n_qudits, dtype=int)
    for i in range(n_qudits):
        x_array[i] = np.random.randint(0, dimensions[i])
        z_array[i] = np.random.randint(0, dimensions[i])
    p_string = PauliString(x_array, z_array, dimensions=dimensions)

    return p_string

def test_to_x():
    target_x = 0
    list_of_failures = []
    for _ in range(20000):
        ps = random_pauli_string([3, 3, 3, 3])
        if ps.n_identities() == 4:
            continue
        c = to_x(ps, 0)
        if c.act(ps).x_exp[target_x] == 0 or c.act(ps).z_exp[target_x] != 0:
            print(f"Failed: {ps} -> {c.act(ps)}")
            list_of_failures.append(ps)

    return list_of_failures


def test_to_ix():
    target_x = 0
    list_of_failures = []
    for _ in range(20000):
        ps = random_pauli_string([3, 3, 3, 3])
        if ps.n_identities() == 4:
            continue
        c = to_ix(ps, 0)
        if c is None:
            print(f"Failed: {ps} -> {c}")
            list_of_failures.append(ps)
            continue
        failed = False
        for i in range(ps.n_qudits()):
            if i == target_x and failed == False:
                if c.act(ps).x_exp[target_x] == 0 or c.act(ps).z_exp[target_x] != 0:
                    print(f"Failed target x: {ps} -> {c.act(ps)}")
                    list_of_failures.append(ps)
                    failed = True
            elif failed == False:
                if c.act(ps).x_exp[i] != 0 or c.act(ps).z_exp[i] != 0:
                    print(f"Failed identity: {ps} -> {c.act(ps)}")
                    list_of_failures.append(ps)
                    failed = True

    return list_of_failures



In [60]:
failures = test_to_x()
print(len(failures))
failures_ix = test_to_ix()
print(len(failures_ix))

0
0


In [ ]:
def remake_basis_select_xz(pauli_sum: PauliSum, x_pauli: int, z_pauli: int, target_qudit: int):
    """
    The goal of this function is to move the pauli_sum to the form (target qudit moved to position 0)
    index    |          pauli         
    x_pauli  | xrz0 x0z0 x0z0 ... x0z0
    z_pauli  | x0zs x0z0 x0z0 ... x0z0
      ...    | x0z0 xrz0 x0z0 ... x0z0
      ...    | x0z0 x0zs x0z0 ... x0z0
      ...    | x0z0 x0z0 xrz0 ... x0z0
    
    
    Until we have a full basis. There are often some skipped rows, due to commutation rules.

    If x_pauli and z_pauli do not anti-commute then we must add a contribution to another pauli (chosen to be 1) that 
    alters the commutation, allowing the desired form of qudit 0.

    """
    
    c = to_ix(pauli_sum[x_pauli], target_qudit)
    # x_pauli now xrz0 x0z0 x0z0 ... x0z0
    if target_qudit != 0:
        c.add_gate(SWAP(target_qudit, 0, pauli_sum.dimensions[target_qudit]))
    pauli_sum = c.act(pauli_sum)

    # now we check the commutation of the two specified paulis - if they anticommute we look for desired form
    # start by making the z_pauli * X I I I I    - useful in either case
    c_temp = to_ix(pauli_sum[z_pauli], 1, ignore=[0])
    pauli_sum = c_temp.act(pauli_sum)
    c += c_temp

    commute = pauli_sum[x_pauli].commute(pauli_sum[z_pauli])
    if commute:
        # we instead make the form 
        # X X I I ...
        # Z Z I I ...
        # We then need a complementary set of paulis to make the basis
        # X Z I I  <-- this one should have the minimal possible coefficient - in principle this still doesn't guarantee
        # Z X I I  |   the minimal weight x components if those outside the basis are dependent on this X 
        #          |   we could catch this case later by going through all paulis to make the X Z out of. 

        raise NotImplementedError('Commutation case not implemented yet')
    
    else:
        # we have anti-commutation
        # first check for Ys on the z_pauli on qudit 0 - remove x components with cnots
        if pauli_sum[z_pauli, 0].z_exp != 0 and pauli_sum[z_pauli, 0].x_exp != 0:
            # z_pauli is YX...I
            print('we have a Y')
            n_cnots = solve_modular_linear(pauli_sum[z_pauli, 0].x_exp, pauli_sum[z_pauli, 1].x_exp,
                                                  pauli_sum.dimensions[0])
            c_temp = Circuit(dimensions=pauli_sum.dimensions)
            for i in range(n_cnots):
                c_temp.add_gate(CX(0, 1, pauli_sum.dimensions[0]))
            pauli_sum = c_temp.act(pauli_sum)
            c += c_temp
            # now should be ZXI...I
        print('now should be ZXI...I')
        print(pauli_sum)

        # we can make the desired form in preamble
        if pauli_sum[z_pauli, 0].z_exp != 0 and pauli_sum[z_pauli, 0].x_exp == 0:
            # z_pauli is ZXI...I 
            n_fourier_sums = solve_modular_linear(pauli_sum[z_pauli, 0].z_exp, pauli_sum[z_pauli, 1].x_exp,
                                                  pauli_sum.dimensions[0])
            c_temp = Circuit(dimensions=pauli_sum.dimensions)
            for i in range(n_fourier_sums):
                c_temp += inv_fourier_sum_fourier(pauli_sum.dimensions, 0, 0, 1)
            pauli_sum = c_temp.act(pauli_sum)
            c += c_temp
            
        else:
            raise Exception("Weird...")
        
    # now we need to clean up the components on the zeroth qudit on the remaining pauli strings that may have been added 
    # by the cnots

    # we make a list of these paulis of the form (index, x_exp, z_exp)
    # list_of_paulis = []
    # for i in range(pauli_sum.n_paulis()):
    #     if i != x_pauli and i != z_pauli:
    #         if pauli_sum[i, 0].x_exp != 0 or pauli_sum[i, 0].z_exp != 0:
    #             list_of_paulis.append((i, pauli_sum[i, 0].x_exp, pauli_sum[i, 0].z_exp))
    
    # # we now loop through the list of paulis and remove the x and z components using cnots and S gates
    # for i, x_exp, z_exp in list_of_paulis:
    #     if x_exp != 0:
    #         # look for an x component in pauli_i and use it to cancel the 0th qudit x
    #         # if there aren't and hadamard a z. 
    #         # (If there are neither we can combine this pauli with the x_pauli or z_pauli, or it is outside the basis)
            

    #     if z_exp == 0:
    #         # same as above but with z
        

    return c
    


In [121]:
ham = random_pauli_hamiltonian(8, [2] * 6, mode='random')
h_red, conditioned_hamiltonians, C, all_phases = pauli_reduce(ham)
# print(conditioned_hamiltonians[0])
leading_x_strings, leading_x_qudit = find_leading_x(conditioned_hamiltonians[0])
print(leading_x_strings, leading_x_qudit)

# ps = ['x1z0 x0z0', 'x1z1 x2z0']
# ps = PauliSum(ps, dimensions=[3, 3])
# print(ps)
# c = inv_fourier_sum_fourier([3, 3], 1, 1, 0)
# cx = Circuit(dimensions=[3, 3])
# cx.add_gate(CX(1, 0, 3))
# c2 = inv_fourier_sum_fourier([3, 3], 0, 0, 1)

# symplectic_effect(c2)


SORTING
[4] 2


In [ ]:
# c = remake_basis_select_x(conditioned_hamiltonians[0], leading_x_strings[0], leading_x_pauli)
# print(c.act(conditioned_hamiltonians[0]))


(-2.108398310953311+4.551911067139117e-17j) |x0z1 x0z0 x0z0 | 0 
(-0.6863371405907298+0j)                    |x1z0 x0z0 x0z0 | 0 
(2.66979710592434-3.26955848014311e-16j)    |x1z0 x0z1 x0z0 | 0 
(-1.0347128199954276+7.84527107294353e-17j) |x0z0 x1z0 x0z0 | 0 
(1.8864886689177587+0j)                     |x0z1 x1z1 x1z0 | 0 
(-0.3968809503619035+4.860389855032647e-17j)|x1z1 x0z0 x0z1 | 0 

now should be ZXI...I
(-2.108398310953311+4.551911067139117e-17j) |x1z0 x0z0 x0z0 | 0 
(-0.6863371405907298+0j)                    |x0z1 x1z0 x0z0 | 0 
(2.66979710592434-3.26955848014311e-16j)    |x0z1 x0z0 x1z0 | 0 
(-1.0347128199954276+7.84527107294353e-17j) |x0z0 x0z0 x0z1 | 0 
(1.8864886689177587+0j)                     |x1z0 x0z0 x1z1 | 0 
(-0.3968809503619035+4.860389855032647e-17j)|x0z1 x1z1 x0z1 | 1 

(-2.108398310953311+4.551911067139117e-17j) |x1z0 x0z0 x0z0 | 0 
(-0.6863371405907298+0j)                    |x0z1 x0z0 x0z0 | 0 
(2.66979710592434-3.26955848014311e-16j)    |x0z1 x1z0 x1z0 | 0 
(

Failed: x2z1 x2z2 x1z1 x1z2  -> x2z2 x2z2 x1z1 x1z2 
Failed: x2z2 x2z1 x2z2 x2z2  -> x2z1 x2z1 x2z2 x2z2 
Failed: x2z1 x2z1 x1z2 x2z2  -> x2z2 x2z1 x1z2 x2z2 
Failed: x2z2 x1z1 x1z2 x1z1  -> x2z1 x1z1 x1z2 x1z1 
Failed: x2z2 x2z1 x1z2 x2z2  -> x2z1 x2z1 x1z2 x2z2 
Failed: x2z1 x2z2 x1z1 x1z1  -> x2z2 x2z2 x1z1 x1z1 
Failed: x2z1 x1z2 x2z2 x1z2  -> x2z2 x1z2 x2z2 x1z2 
Failed: x2z2 x1z1 x1z2 x1z2  -> x2z1 x1z1 x1z2 x1z2 
Failed: x2z2 x2z2 x2z2 x1z1  -> x2z1 x2z2 x2z2 x1z1 
Failed: x2z1 x1z1 x2z2 x1z2  -> x2z2 x1z1 x2z2 x1z2 
Failed: x2z2 x2z1 x2z2 x2z2  -> x2z1 x2z1 x2z2 x2z2 
Failed: x2z1 x2z2 x2z2 x2z1  -> x2z2 x2z2 x2z2 x2z1 
Failed: x2z2 x2z2 x2z2 x2z1  -> x2z1 x2z2 x2z2 x2z1 
Failed: x2z1 x1z1 x2z2 x2z1  -> x2z2 x1z1 x2z2 x2z1 
Failed: x2z2 x2z2 x1z2 x1z1  -> x2z1 x2z2 x1z2 x1z1 
Failed: x2z2 x1z1 x2z2 x2z1  -> x2z1 x1z1 x2z2 x2z1 
Failed: x2z1 x2z2 x2z1 x2z1  -> x2z2 x2z2 x2z1 x2z1 
Failed: x2z1 x2z2 x1z1 x2z2  -> x2z2 x2z2 x1z1 x2z2 
Failed: x2z2 x1z1 x2z2 x1z2  -> x2z1 x1z1 x2z2